## Building LightWeight UNet Architecture

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 7, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 7, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p


def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(7, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model


input_shape = (512, 512, 7)
model = build_unet(input_shape)
model.summary()

## Creating Features and Inputs

In [28]:
import os
import cv2
import numpy as np
X = []
y = []
i = 10
for elem in os.listdir('ground_truth/'):
    curr_image = []
    for image in os.listdir(f'ground_truth/{elem}/synthetic_cloud_images/'):
        synthetic_image_band = cv2.imread(f'ground_truth/{elem}/synthetic_cloud_images/{image}',cv2.IMREAD_GRAYSCALE)
        curr_image.append(synthetic_image_band)
    ans = np.dstack([elem for elem in curr_image])
    X.append(ans)
    curr_image = []
    for image in os.listdir(f'ground_truth/{elem}/original_image/'):
        original_image = cv2.imread(f'ground_truth/{elem}/original_image/{image}',cv2.IMREAD_GRAYSCALE)
        curr_image.append(original_image)
    ans = np.dstack([elem for elem in curr_image])
    y.append(ans)
    i -= 1
    if i == 0 :break

In [29]:
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)

(10, 512, 512, 7)
(10, 512, 512, 7)


In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y,test_size = 0.25)

In [ ]:
model.compile(loss="mse",optimizer="adam",metrics=["Accuracy"])
model.fit(X,y,epochs=200,batch_size = 10, validation_data=(X_test,y_test))